# Train a selector of the relevant ToS clauses

## Load libraries

In [24]:
import requests
import json
import concurrent.futures
from rich import print as rprint
from rich.tree import Tree
from rich.panel import Panel
from rich.console import Console
from rich.progress import (
    Progress,
    SpinnerColumn,
    BarColumn,
    TextColumn,
    TimeRemainingColumn,
    MofNCompleteColumn
)
from pathlib import Path
from langdetect import detect, DetectorFactory

## Global variables

In [25]:
console = Console()
ROOT = Path("../..")
DATA_DIR = ROOT / "data" / "TOSDR"
OUTPUT_DIR = DATA_DIR / "scraped_v2.json"

## Utilities functions

In [26]:
def visualize_rating(item):
    """Visualizes the dataset item."""
    if not item: return
    title = f"[bold white]{item['service']}[/bold white] (Rating: [bold yellow]{item['rating']}[/bold yellow])"
    tree = Tree(title)
    
    doc_node = tree.add("[bold blue]📄 Documents (Source Text)[/bold blue]")
    for d in item['docs']:
        doc_node.add(f"[cyan]{d['title']}[/cyan] ({len(d['text'])} chars) - [link={d.get('url','')}]Source[/link])")
                
        
    point_node = tree.add("[bold magenta]💡 Analysis Points[/bold magenta]")
    for p in item['summaries']:
        color = "green" if p['sentiment'] == "good" else "red" if p['sentiment'] == "bad" else "white"
        point_node.add(f"[{color}]• {p['title']}[/{color}]")
    
    rprint(Panel(tree, border_style="blue"))

## Processing

In [27]:
with open(OUTPUT_DIR, 'r', encoding='utf-8') as f:
    final_dataset = json.load(f)

for item in final_dataset[:2]:
    visualize_rating(item)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ UNRIC (Rating: N/A)                                                                                             │
│ ├── 📄 Documents (Source Text)                                                                                  │
│ │   └── Privacy notice (798 chars) - ]8;id=170926;https://unric.org/en/privacy-notice/\Source]8;;\)                                                                    │
│ └── 💡 Analysis Points                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Tory Burch LLC (Rating: N/A)                                                                                    │
│ ├── 📄 Documents (Source Text)                                                                                  │
│ │   ├── Terms of Use (12618 chars) - ]8;id=240270;https://www.toryburch.com/en-us/terms-of-use/content-terms-and-conditions/\Source]8;;\)                                                                    │
│ │   └── Privacy Policy (36715 chars) - ]8;id=466708;https://www.toryburch.com/en-us/content-privacy/content-privacy/\Source]8;;\)                                                                  │
│ └── 💡 Analysis Points                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [28]:
for item in final_dataset[:10]:
    title = f"[bold white]{item['service']}[/bold white] (Rating: [bold yellow]{item['rating']}[/bold yellow])"
    rprint(title)
    for d in item['docs']:
       title_of_doc = f"Document: [cyan]{d['title']}[/cyan]"
       source_url = d.get('url','')
       rprint(title_of_doc)
       if source_url:
        rprint(f"[link={source_url}]Source URL[/link]")
       sample_text = d['text'][:500].replace('\n', ' ') + "..."
       lang = detect(d['text'])
       rprint(f"[dim]Detected Language: {lang}[/dim]")
       rprint(f"[dim]{sample_text}[/dim]")

UNRIC (Rating: N/A)

Document: Privacy notice

]8;id=179524;https://unric.org/en/privacy-notice/\Source URL]8;;\

Detected Language: en

Privacy Notice 04/11/2019 FacebookTwitterWhatsAppEmailPrint    UNITED NATIONS’ PRIVACY NOTICE  By accessing this 
site, certain information about the User, such as Internet protocol (IP) addresses, navigation through the Site, 
the software used and the time spent, along with other similar information, will be stored on United Nations 
servers. These will not specifically identify the User.  The information will be used internally only for web site 
traffic analysis. If the User provides unique iden...

...

Cookie Notice  We use cookies for reasons including website functionality and traffic analytics. Please see the 
"Cookies Notice" section of our Website Privacy Policy for a more detailed description and click on "Cookie 
Settings" to customize how our site uses cookies for you. Website Cookies Notice  CloseAccept CookiesCookie 
Settings  * Cookie Settings * Strictly Necessary Cookies * Performance Cookies * Functional Cookies * Targeting 
Cookies * More Information  Privacy Preference Center  Activ...

Document: Product Privacy Policy

]8;id=423148;https://www.goguardian.com/productPrivacy.html\Source URL]8;;\

Detected Language: en

Cookie Notice  We use cookies for reasons including website functionality and traffic analytics. Please see the 
"Cookies Notice" section of our Website Privacy Policy for a more detailed description and click on "Cookie 
Settings" to customize how our site uses cookies for you. Website Cookies Notice  CloseAccept CookiesCookie 
Settings  * Cookie Settings * Strictly Necessary Cookies * Performance Cookies * Functional Cookies * Targeting 
Cookies * More Information  Privacy Preference Center  Activ...

In [29]:
# compute language distribution
lang_counts = {}
for item in final_dataset:
    for d in item['docs']:
        try:
            lang = detect(d['text'])
            if lang not in lang_counts:
                lang_counts[lang] = 0
            lang_counts[lang] += 1
        except:
            continue
rprint("[bold underline]Language Distribution in Documents:[/bold underline]")
for lang, count in lang_counts.items():
    rprint(f"[bold]{lang}[/bold]: {count} documents")

Language Distribution in Documents:

en: 353 documents

nl: 3 documents

de: 7 documents

fr: 7 documents

...

pt: 1 documents

zh-cn: 7 documents

es: 1 documents

vi: 1 documents

sv: 1 documents